In [ ]:
# ===================================================================
# IMPORTS AND ENVIRONMENT SETUP
# ===================================================================
# This notebook demonstrates an advanced CrewAI workflow for AI-powered image generation
# We'll create a multi-agent system that:
# 1. Enhances text prompts to be more descriptive
# 2. Generates images using DALL-E based on enhanced prompts
# 3. Downloads and saves generated images locally

# Core CrewAI imports for building multi-agent systems
from crewai import Crew, Agent, Task, LLM

# Pre-built tool for DALL-E image generation
from crewai_tools import DallETool

# Base class for creating custom tools
from crewai.tools import BaseTool

# Pydantic for defining input schemas and data validation
from pydantic import BaseModel, Field

# Type hints for better code documentation
from typing import Type, Any

# HTTP requests library for downloading images
import requests

# OS utilities for file operations
import os

# Load environment variables (API keys) from .env file
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
# ===================================================================
# LANGUAGE MODEL CONFIGURATION
# ===================================================================
# Initialize the Large Language Model (LLM) that will power our AI agents
# GPT-4 is used here for its superior reasoning and creative capabilities
# This LLM will be assigned to agents to help them understand tasks and make decisions

llm = LLM(model="gpt-4")

In [ ]:
# ===================================================================
# CUSTOM TOOL INPUT SCHEMA
# ===================================================================
# Define the input schema for our custom Image Downloader tool
# Pydantic BaseModel ensures type safety and automatic validation
# This tells the AI agent what parameters the tool expects

class DownloadImageInput(BaseModel):
    """
    Input schema for the Image Downloader tool.
    Defines what information the agent needs to provide to download an image.
    """
    # The URL of the image to download (required field)
    image_url: str = Field(..., description="The URL of the image to download")

In [ ]:
# ===================================================================
# CUSTOM TOOL IMPLEMENTATION
# ===================================================================
# Create a custom tool by extending BaseTool
# This tool will download images from URLs and save them locally
# Custom tools allow you to extend CrewAI's capabilities beyond built-in tools

class DownloadImageTool(BaseTool):
    """
    Custom tool for downloading images from URLs.
    This demonstrates how to create reusable tools for your AI agents.
    """
    # Tool name that agents will see and use
    name: str = "Image Downloader Tool"
    
    # Description helps the AI agent understand when to use this tool
    description: str = "Downloads image from url"
    
    # Link the input schema to enforce proper arguments
    args_schema: Type[BaseModel] = DownloadImageInput

    def _run(self, image_url: str) -> str:
        """
        Main execution method that performs the actual download.
        
        Args:
            image_url: The URL of the image to download
            
        Returns:
            Success message with filename or error message
        """
        # Extract filename from URL and add .png extension
        filename = f"{os.path.basename(image_url)}.png"

        try:
            # Make HTTP GET request with streaming enabled for large files
            response = requests.get(image_url, stream=True)
            # Raise exception if request failed (4xx or 5xx status codes)
            response.raise_for_status()
        except requests.RequestException as e:
            return f"Failed {e}"

        # Write image data to file in binary mode
        with open(filename, "wb") as image_file:
            # Download in chunks to handle large files efficiently
            for chunk in response.iter_content(chunk_size=8192):
                image_file.write(chunk)

        return f"Save as {filename}"

In [ ]:
# ===================================================================
# DALL-E TOOL CONFIGURATION
# ===================================================================
# Configure the DALL-E tool with specific parameters
# This is a pre-built CrewAI tool that interfaces with OpenAI's DALL-E API

dalle_tool = DallETool(
    model="dall-e-3",        # Use DALL-E 3 model (latest version with best quality)
    size="1024x1024",        # Generate square images at 1024x1024 resolution
    quality="standard",      # Quality setting: "standard" or "hd" (hd costs more)
    n=1                      # Number of images to generate per request
)

In [ ]:
# ===================================================================
# AGENT 1: PROMPT IMPROVER
# ===================================================================
# First agent in our workflow - specializes in enhancing simple prompts
# This agent takes basic descriptions and makes them rich, detailed, and vivid
# Good prompts lead to better image generation results

prompt_improver_agent = Agent(
    llm=llm,                                                              # Assign GPT-4 to power this agent
    role="Prompt improver",                                               # The agent's role/identity
    backstory="Ai agent that enhances prompts to be more detailed",       # Background context for the agent
    goal="Improve text prompts for optimal image generation",             # The agent's primary objective
    verbose=True                                                          # Enable detailed logging to see the agent's thought process
)

In [ ]:
# ===================================================================
# AGENT 2: IMAGE GENERATOR
# ===================================================================
# Second agent in our workflow - specializes in image generation and management
# This agent has access to tools (DALL-E and Image Downloader) to accomplish its tasks
# It receives the enhanced prompt from Agent 1 and produces the final image

image_generator_agent = Agent(
    llm=llm,                                                                                        # Assign GPT-4 to power this agent
    role="Image generator",                                                                         # The agent's role/identity
    backstory="An AI agent that generates images based on detailed prompts, using DALLE and save the image in the current directory",  # Detailed background
    goal="Generate images from enhanced prompts using DALLE and save them locally",                 # The agent's primary objective
    tools=[dalle_tool, DownloadImageTool()],                                                        # Equip agent with necessary tools
    verbose=True                                                                                    # Enable detailed logging
)

In [ ]:
# ===================================================================
# TASK DEFINITIONS
# ===================================================================
# Define the sequence of tasks that need to be accomplished
# Tasks are assigned to specific agents and executed in order
# Each task's output becomes context for subsequent tasks

# TASK 1: Enhance the user's prompt
# Assigned to: prompt_improver_agent
# Input: Simple text prompt from user
# Output: Rich, detailed description suitable for image generation
enhance_prompt_task = Task(
    description="Improve this prompt ({initial_prompt}) to make it more descriptive and detailed for image generation.",  # Task instruction with variable placeholder
    expected_output='"image_description": "Enhanced Prompt here',                                                         # What the agent should produce
    agent=prompt_improver_agent                                                                                           # Which agent handles this task
)

# TASK 2: Generate image using DALL-E
# Assigned to: image_generator_agent
# Input: Enhanced prompt from Task 1
# Output: URL of the generated image
generate_image_task = Task(
    description="Generate an image from the enhanced prompt using DALLE",    # Task instruction
    expected_output="image_url: URL of the generated image",                 # What the agent should produce
    agent=image_generator_agent                                               # Which agent handles this task
)

# TASK 3: Download and save the image
# Assigned to: image_generator_agent
# Input: Image URL from Task 2
# Output: Local file saved to disk
download_image_task = Task(
    description="Download the generated image from the provided URL",        # Task instruction
    expected_output="Image downloaded and saved locally",                    # What the agent should produce
    agent=image_generator_agent                                               # Which agent handles this task
)

In [ ]:
# ===================================================================
# CREW ASSEMBLY
# ===================================================================
# Assemble the crew by combining agents and tasks
# The Crew orchestrates the workflow, ensuring tasks are executed in sequence
# and agents collaborate to achieve the overall goal

crew = Crew(
    agents=[prompt_improver_agent, image_generator_agent],                  # List of all agents in this crew
    tasks=[enhance_prompt_task, generate_image_task, download_image_task]   # List of tasks in execution order
)

# WORKFLOW SUMMARY:
# 1. User provides a simple prompt (e.g., "A futuristic cityscape at sunset")
# 2. Prompt Improver Agent enhances it with rich details
# 3. Image Generator Agent creates the image using DALL-E
# 4. Image Generator Agent downloads and saves the image locally

In [ ]:
# ===================================================================
# EXECUTE THE CREW
# ===================================================================
# Kickoff the crew execution with input parameters
# The 'inputs' dictionary provides values for variables referenced in task descriptions
# In this case, {initial_prompt} in enhance_prompt_task gets replaced with the value below

# Example execution with a simple prompt
result = crew.kickoff(inputs={'initial_prompt': 'A futuristic cityscape at sunset'})

# The result object contains:
# - raw: The final output text from the last task
# - tasks_output: Detailed output from each individual task
# - token_usage: Information about API token consumption

# TRY IT YOURSELF:
# Change the initial_prompt to generate different images:
# - 'A serene mountain lake at dawn'
# - 'A bustling marketplace in a cyberpunk city'
# - 'An ancient library filled with magical books'

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Prompt improver                                                                                         │
│                                                                                                                 │
│  Task: Improve this prompt (A futuristic cityscape at sunset) to make it more descriptive and detailed for      │
│  image generation.                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Prompt improver                                                                                         │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  "image_description": "A sprawling, futuristic cityscape bathed in the golden glow of a majestic sunset.        │
│  Multitudes of towering skyscrapers, with sleek designs and streamlined shapes, reach for the cloud-pocked      │
│  sky. Structures of various, startling architectural designs are adorned with a myriad of gleaming neon lights  │
│  in a symphony of colors, all reflecting off the surfaces of the buildings, creating a breathtaking mosaic of   │
│  light. The sun, a fiery orb, is slowly sinking at the horizon, painting the sky with warm hues of coppers and  │
│  purples. In the foreground, a monorail streaks along its track, weaving through the city, while far up above   │
│  them, advanced aircrafts dart between buildings and over the city, leaving contrails in their paths. The city  │
│  is surrounded by nature, the edge of the cityscape giving way to the verdant lushness of carefully maintained  │
│  parks and forests. In the distance, pristine bodies of water glitter under the sunset, completing this         │
│  harmonious blend of technologic advancement and nature's beauty."                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Image generator                                                                                         │
│                                                                                                                 │
│  Task: Generate an image from the enhanced prompt using DALLE                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Image generator                                                                                         │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  The task requires me to generate an image based on an enhanced prompt using Dall-E. The image_description      │
│  provided is detailed and depicts a futuristic cityscape at sunset.                                             │
│  Action:                                                                                                        │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Image generator                                                                                         │
│                                                                                                                 │
│  Task: Download the generated image from the provided URL                                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Image generator                                                                                         │
│                                                                                                                 │
│  Thought: Thought: Now that I have the url of the generated image, I need to download the image using the       │
│  Image Downloader Tool and save it locally.                                                                     │
│                                                                                                                 │
│  Using Tool: Image Downloader Tool                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  {                                                                                                              │
│    "image_url":                                                                                                 │
│  "https://oaidalleapiprodscus.blob.core.windows.net/private/org-S87Jqg5ocbUr83IVDwKuwC46/user-W3S7A7WBb4fiQX0p  │
│  jfktBQWt/img-Jyz6bbtYJv3tmsMkq3pmQvu5.png?st=2025-12-31T01%3A57%3A42Z&se=2025-12-31T03%3A57%3A42Z&sp=r&sv=202  │
│  4-08-04&sr=b&rscd=inline&rsct=image/png&skoid=c4071b20-fbe5-4cbe-8e48-4476b70c6a90&sktid=a48cca56-e6da-484e-a  │
│  814-9c849652bcb3&skt=2025-12-30T20%3A10%3A16Z&ske=2025-12-31T20%3A10%3A16Z&sks=b&skv=2024-08-04&sig=dSaTHkM1N  │
│  no/y83kOKdOTnflZv29RU%2Bi1L7a3Gm3ax4%3D"                                                                       │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Save as y83kOKdOTnflZv29RU%2Bi1L7a3Gm3ax4%3D.png                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Image generator                                                                                         │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  The image has been successfully downloaded and saved as "y83kOKdOTnflZv29RU%2Bi1L7a3Gm3ax4%3D.png" in the      │
│  current directory.                                                                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

CrewOutput(raw='The image has been successfully downloaded and saved as "y83kOKdOTnflZv29RU%2Bi1L7a3Gm3ax4%3D.png" in the current directory.', pydantic=None, json_dict=None, tasks_output=[TaskOutput(description='Improve this prompt (A futuristic cityscape at sunset) to make it more descriptive and detailed for image generation.', name='Improve this prompt (A futuristic cityscape at sunset) to make it more descriptive and detailed for image generation.', expected_output='"image_description": "Enhanced Prompt here', summary='Improve this prompt (A futuristic cityscape at sunset) to make...', raw='"image_description": "A sprawling, futuristic cityscape bathed in the golden glow of a majestic sunset. Multitudes of towering skyscrapers, with sleek designs and streamlined shapes, reach for the cloud-pocked sky. Structures of various, startling architectural designs are adorned with a myriad of gleaming neon lights in a symphony of colors, all reflecting off the surfaces of the buildings, cre

In [ ]:
# ===================================================================
# ADDITIONAL EXPLORATION (OPTIONAL)
# ===================================================================
# Use this cell to experiment with the workflow:

# 1. View individual task outputs:
# print(result.tasks_output[0].raw)  # Prompt improver output
# print(result.tasks_output[1].raw)  # Image generator output
# print(result.tasks_output[2].raw)  # Download task output

# 2. Check token usage and cost:
# print(f"Total tokens used: {result.token_usage.total_tokens}")
# print(f"Prompt tokens: {result.token_usage.prompt_tokens}")
# print(f"Completion tokens: {result.token_usage.completion_tokens}")

# 3. Experiment with DALL-E parameters:
# Try different sizes: "1024x1024", "1792x1024", "1024x1792"
# Try quality: "standard" vs "hd"

# 4. Extend the workflow:
# - Add an agent to critique the generated image
# - Create multiple variations of the same prompt
# - Add image post-processing capabilities

# ===================================================================
# IMPORTANT OBSERVATION ABOUT EXECUTION:
# ===================================================================
# If you examine the execution output above, you may notice something interesting:
# 
# The generate_image_task (Task 2) didn't actually generate the image!
# Instead, the download_image_task (Task 3) both generated AND downloaded the image.
#
# WHY THIS HAPPENS:
# - Task 2's expected_output was too vague, so the agent didn't understand it 
#   needed to actually call the DALL-E tool
# - The agent acknowledged the task but didn't execute it
# - Task 3's agent was smart enough to realize no image URL existed yet
# - So it autonomously completed the missing step (generate) before downloading
#
# This demonstrates both:
# ✓ Agent autonomy: Agents can adapt and complete missing prerequisites
# ✗ Task clarity importance: Vague instructions can lead to unexpected behavior
#
# TO IMPROVE THIS WORKFLOW:
# Make Task 2's expected output more explicit:
# expected_output="A complete image URL generated by DALL-E tool. You MUST use the Dall-E Tool to generate the image."

# KEY LEARNING POINTS:
# ✓ Multi-agent collaboration: Agents work together on complex tasks
# ✓ Custom tools: Extend functionality beyond built-in capabilities
# ✓ Sequential workflow: Output from one task feeds into the next
# ✓ Tool usage: Agents intelligently select and use appropriate tools
# ✓ Agent autonomy: Agents can adapt when prerequisites are missing
# ✓ Task clarity: Clear, explicit instructions prevent unexpected behavior
# ✓ Real-world application: Practical AI workflow for content creation


In [13]:
# ===================================================================
# IMPROVED VERSION: Fix the Task Execution Issue
# ===================================================================
# Uncomment and run this cell to see the corrected workflow

# Create improved task definitions with more explicit instructions
enhance_prompt_task_v2 = Task(
    description="Improve this prompt ({initial_prompt}) to make it more descriptive and detailed for image generation.",
    expected_output='A detailed, enhanced image description in the format: "image_description": "Your enhanced prompt here"',
    agent=prompt_improver_agent
)

generate_image_task_v2 = Task(
    description="""Use the Dall-E Tool to generate an image from the enhanced prompt. 
    You MUST call the Dall-E Tool with the enhanced description from the previous task. 
    Do not just acknowledge - actually execute the tool.""",
    expected_output="The complete image URL returned by DALL-E after successful generation. Format: image_url: [URL]",
    agent=image_generator_agent,
    context=[enhance_prompt_task_v2]  # Explicitly link to previous task
)

download_image_task_v2 = Task(
    description="""Download the image from the URL provided by the previous task. 
    Use the Image Downloader Tool to save it locally.""",
    expected_output="Confirmation message with the local filename where image was saved",
    agent=image_generator_agent,
    context=[generate_image_task_v2]  # Explicitly link to previous task
)

# Create improved crew
crew_v2 = Crew(
    agents=[prompt_improver_agent, image_generator_agent],
    tasks=[enhance_prompt_task_v2, generate_image_task_v2, download_image_task_v2],
    verbose=True
)

# Execute improved workflow
result_v2 = crew_v2.kickoff(inputs={'initial_prompt': 'A magical forest with glowing mushrooms'})

# IMPROVEMENTS MADE:
# 1. More detailed task descriptions that explicitly mention tool usage
# 2. Clear expected_output formats so agents know exactly what to produce
# 3. Added 'context' parameter to explicitly link tasks (task dependencies)
# 4. Used imperative language: "You MUST call the tool" vs "Generate an image"

# ===================================================================
# IMPORTANT: WHY THIS "IMPROVED" VERSION ALSO FAILED
# ===================================================================
# If you ran this cell, you'll notice it STILL didn't work properly!
#
# WHAT WENT WRONG:
# 1. Task 2 still said "Action:" but didn't execute DALL-E tool
# 2. Task 3 couldn't find the image description from Task 1
# 3. No image was actually generated or downloaded
#
# ROOT CAUSES:
# • Agent Prompt Template Conflict: The agent's internal prompt format conflicts
#   with task instructions, causing it to acknowledge but not execute
# • Context Chain Breaking: When Task 2 fails, Task 3 loses access to Task 1's output
# • Tool Execution Hesitation: The LLM is being overly cautious about executing tools
#
# BETTER SOLUTIONS:
# Option 1: Combine tasks - Have ONE task do both generation and download
# Option 2: Use async=False and force sequential execution with stricter validation
# Option 3: Create a single agent that handles the entire workflow
# Option 4: Use CrewAI's newer features like Flows for better control
# Option 5: Add output_file parameter to force agents to save their results
#
# This demonstrates an important lesson:
# ⚠️ Even with explicit instructions, AI agents can still behave unexpectedly
# ⚠️ Task decomposition isn't always better - sometimes simpler is more reliable
# ⚠️ Context passing between tasks can be fragile in multi-step workflows


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: c0ceedc6-99fd-458e-b997-0d381da4d2c5                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Prompt improver                                                                                         │
│                                                                                                                 │
│  Task: Improve this prompt (A magical forest with glowing mushrooms) to make it more descriptive and detailed   │
│  for image generation.                                                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Prompt improver                                                                                         │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  "image_description": "A mystical forest densely populated with towering ancient trees, their twisted roots     │
│  carpeted with emerald moss. A mixture of deep shades of forest greens and dark browns, with beams of ethereal  │
│  moonlight peeking through the intricate lattice of leaves above. Scattered throughout the undergrowth,         │
│  clusters of oversized, luminescent mushrooms cast an enchanting glow. Their caps vary in colors from vibrant   │
│  blues to purples, giving off an otherworldly iridescent light. This magical forest scene is bathed in an       │
│  atmospheric, enchanting sheen."                                                                                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

...

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Image generator                                                                                         │
│                                                                                                                 │
│  Thought: The task requires me to download an image from a given URL and save it locally using the Image        │
│  Downloader Tool. To execute this task, I will use the provided URL as an input.                                │
│                                                                                                                 │
│  Using Tool: Image Downloader Tool                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Image generator                                                                                         │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  The image has been downloaded and saved locally with the filename                                              │
│  png&skoid=52f8f7b3-ca8d-4b21-9807-8b9df114d84c&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2025-12-30T21%3  │
│  A34%3A27Z&ske=2025-12-31T21%3A34%3A27Z&sks=b&skv=2024-08-04&sig=S9o%2BK9SaE8avwmL6HQstjBEIwdrqzhm3zT0796VJXDI  │
│  %3D.png                                                                                                        │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 1c75a794-ea4c-4af0-bcbc-f5c851d32fe1                                                                     │
│  Agent: Image generator                                                                                         │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: c0ceedc6-99fd-458e-b997-0d381da4d2c5                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: The image has been downloaded and saved locally with the filename                                │
│  png&skoid=52f8f7b3-ca8d-4b21-9807-8b9df114d84c&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2025-12-30T21%3  │
│  A34%3A27Z&ske=2025-12-31T21%3A34%3A27Z&sks=b&skv=2024-08-04&sig=S9o%2BK9SaE8avwmL6HQstjBEIwdrqzhm3zT0796VJXDI  │
│  %3D.png                                                                                                        │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Tracing Status ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing is disabled.                                                                                     │
│                                                                                                                 │
│  To enable tracing, do any one of these:                                                                        │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
# ===================================================================
# WORKING SOLUTION: Simplified Approach
# ===================================================================
# Instead of splitting into 3 tasks, use 2 simpler tasks that actually work

# # Task 1: Enhance prompt (same as before)
# enhance_prompt_working = Task(
#     description="Improve this prompt ({initial_prompt}) to make it more descriptive and detailed for image generation.",
#     expected_output='Enhanced image description',
#     agent=prompt_improver_agent
# )

# # Task 2: COMBINED - Generate AND Download (everything in one task)
# generate_and_download_working = Task(
#     description="""Using the enhanced prompt from the previous task:
#     1. First, use the Dall-E Tool to generate an image
#     2. Then, use the Image Downloader Tool to save the generated image locally
#     Complete both steps in this single task.""",
#     expected_output="Confirmation that image was generated and saved with the filename",
#     agent=image_generator_agent,
#     context=[enhance_prompt_working]
# )

# # Create working crew with simplified workflow
# crew_working = Crew(
#     agents=[prompt_improver_agent, image_generator_agent],
#     tasks=[enhance_prompt_working, generate_and_download_working],
#     verbose=True
# )

# # Execute working workflow
# result_working = crew_working.kickoff(inputs={'initial_prompt': 'A cyberpunk street market at night'})
# print("\n✅ SUCCESS! Check your directory for the downloaded image.")

# WHY THIS WORKS BETTER:
# ✓ Fewer task transitions = less chance for context loss
# ✓ Single agent handles related operations together
# ✓ Clearer goal: "Do both X and Y" vs separate "Do X" then "Do Y"
# ✓ Agent maintains context within one execution
# ✓ More robust to agent hesitation issues

# KEY INSIGHT:
# Sometimes the best solution is SIMPLER, not MORE explicit.
# Combining related tasks reduces coordination overhead.


In [ ]:
# ===================================================================
# SUMMARY: Lessons Learned from This Notebook
# ===================================================================

# This notebook demonstrates several important concepts in AI agent workflows:

# 📊 WHAT WE LEARNED:

# 1. TASK GRANULARITY MATTERS
#    • Too many small tasks → Context loss, coordination overhead
#    • Original: 3 tasks → Tasks 2&3 both failed to execute properly
#    • Better: 2 tasks with combined operations

# 2. AGENT BEHAVIOR IS UNPREDICTABLE
#    • Even explicit instructions ("You MUST...") don't guarantee execution
#    • Agents may acknowledge tasks without completing them
#    • LLM decision-making can be conservative about tool usage

# 3. CONTEXT PASSING IS FRAGILE
#    • The 'context' parameter should link tasks, but can break
#    • If Task N fails, Task N+1 loses access to earlier outputs
#    • Sequential dependencies create single points of failure

# 4. SIMPLICITY > COMPLEXITY
#    • More explicit instructions ≠ Better results
#    • Combining related operations often works better
#    • Fewer transitions = fewer opportunities for failure

# 📋 DESIGN PATTERNS FOR RELIABLE AI WORKFLOWS:

# Pattern 1: Combine Related Tasks
# ✅ DO: "Generate and download image"
# ❌ DON'T: "Generate image" → "Download image" as separate tasks

# Pattern 2: Single Agent for Related Operations  
# ✅ DO: One agent with both DALL-E and Downloader tools
# ❌ DON'T: Separate agents for generation vs download

# Pattern 3: Clear, Actionable Instructions
# ✅ DO: "Use Tool X to do Y, then use Tool Z to do W"
# ❌ DON'T: "You should do Y" (vague, no explicit tool mention)

# Pattern 4: Validate Intermediate Steps
# ✅ DO: Check that each step completed before proceeding
# ❌ DON'T: Assume tasks completed successfully

# 🔬 DEBUGGING AI AGENT ISSUES:

# When agents don't behave as expected:
# 1. Check agent output - did they acknowledge but not execute?
# 2. Verify context passing - can tasks see previous outputs?
# 3. Simplify the workflow - combine tasks, reduce transitions
# 4. Test tools independently - do they work when called directly?
# 5. Add explicit output validation - force agents to produce results

# 🎯 PRACTICAL TAKEAWAYS:

# • AI agent systems are still emerging technology with quirks
# • Always have fallback strategies when designing workflows
# • Test thoroughly - "should work" ≠ "does work"
# • Monitor execution logs to understand agent decision-making
# • Balance between task decomposition and execution reliability

# This notebook serves as both:
# • A working example of AI agent image generation
# • A case study in real-world AI engineering challenges

print("Notebook complete! You've learned about CrewAI agent behaviors, "
      "task design patterns, and common pitfalls to avoid.")
